In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# Descargar los datos
df = pd.read_csv('amzn.us.txt')
df = df[['Date', 'Open']]
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Normalizar los datos
scaler = MinMaxScaler()
df['Open'] = scaler.fit_transform(df[['Open']])

# Función para dividir los datos de entrenamiento y testeo
def train_test_split(data, train_size):
    train_index = int(len(data) * train_size)
    train_data, test_data = data[:train_index], data[train_index:]
    return train_data, test_data

# Función para crear secuencias de datos para el modelo RNN
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Parámetros
seq_length = 60  # longitud de la secuencia de entrada
train_size_initial = 0.5  # tamaño inicial del conjunto de entrenamiento
train_size_increment = 0.1  # incremento en el tamaño del conjunto de entrenamiento

# Definir el modelo
def create_model():
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model
    
# Ejecutar el proceso 20 veces
for iteration in range(20):
    print(f"Running iteration {iteration + 1}...")

    mse_train = []
    mse_test = []

    for i in range(5):  # 5 iteraciones para 50%, 60%, ..., 90% de los datos
        train_size = train_size_initial + i * train_size_increment
        if train_size > 0.9:
            break

        # Tamaño del conjunto de entrenamiento y conjunto para la predicción
        train_index_end = int(train_size * len(df))
        test_index_start = train_index_end
        test_index_end = int((train_size + train_size_increment) * len(df))
        if test_index_end > len(df):
            test_index_end = len(df)

        train_start_date = df.index[0]
        train_end_date = df.index[train_index_end - 1]
        test_start_date = df.index[test_index_start] if test_index_start < len(df) else df.index[-1]
        test_end_date = df.index[test_index_end - 1] if test_index_end <= len(df) else df.index[-1]

        # Crear el modelo
        model = create_model()

        # Entrenar el modelo
        train_data, _ = train_test_split(df['Open'].values, train_size)
        X_train, y_train = create_sequences(train_data, seq_length)
        if len(X_train) == 0:
            continue
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        # Calcular el error cuadrático medio en el conjunto de entrenamiento
        y_train_pred = model.predict(X_train)
        mse_train.append(mean_squared_error(y_train, y_train_pred))

        # Realizar la predicción en el siguiente conjunto de prueba
        test_data = df['Open'].values[test_index_start:test_index_end]
        X_test, y_test = create_sequences(test_data, seq_length)
        if len(X_test) == 0:
            continue
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        y_pred = model.predict(X_test)
        mse_test.append(mean_squared_error(y_test, y_pred))

        # Guardar y_pred rescaled en el archivo correspondiente
        y_pred_rescaled = scaler.inverse_transform(y_pred).flatten()
        y_pred_str = ",".join(map(str, y_pred_rescaled))
        with open(f'y_pred_rescaled_part_{i + 1}_LSTM.csv', 'a') as f:
            f.write(f"Iteration {iteration + 1},{y_pred_str}\n")

    # Guardar los resultados de las métricas
    with open('train_mse_LSTM.csv', 'a') as f:
        f.write(f"Iteration {iteration + 1}," + ",".join(map(str, mse_train)) + "\n")

    with open('test_mse_LSTM.csv', 'a') as f:
        f.write(f"Iteration {iteration + 1}," + ",".join(map(str, mse_test)) + "\n")

print("All iterations completed.")


2024-06-02 18:52:07.684498: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Running iteration 1...


2024-06-02 18:52:11.891494: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-02 18:52:11.961168: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-02 18:52:11.961234: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-02 18:52:11.984637: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-02 18:52:11.984707: I external/local_xla/xla/stream_executor

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
All iterations completed.
